In this notebook:
- Convert the IAAF points tables to tidy pandas dataframe.
- Will use this to add a view of the comparative strengths of the various joggling records.
- Note that this will not fill the gap of comparing records across different props (e.g. 3b vs, 5b)

In [1]:
import pandas as pd
import re
import numpy as np

## PyPDF2

In [2]:
from PyPDF2 import PdfReader
reader = PdfReader('points_data_pdf.pdf')

In [4]:
len(reader.pages)

# M Sprints 9-36
# M Middle Distance 39-66
# M Long Distance 69-96
# M Road Running 99-126

# W Sprints 189-216
# W Middle Distance 219-246
# W Long Distance 249-276
# W Road Running 279-306

# Example String:
reader.pages[189].extract_text()

'WOMEN’S SPRINTS, HURDLES AND RELAYS / FEMMES SPRINTS, HAIES ET RELAIS \n182  Points 100m 100mH 200m 300m 400m 400mH 4x100m 4x200m 4x400m  1400 10.12 11.24 20.51 32.27 45.35 48.07 38.04 1:19.29 3:00.61  1399  - 11.25 20.52 32.29 45.37 48.09 38.06 1:19.34 3:00.72  1398  -  -  - 32.31 45.40 48.12 38.08 1:19.39 3:00.83  1397 10.13 11.26 20.53 32.32 45.42 48.15 38.11 1:19.43 3:00.94  1396  - 11.27 20.54 32.34 45.44 48.18 38.13 1:19.48 3:01.04  1395 10.14  - 20.55 32.35 45.46 48.21 38.15 1:19.53 3:01.15  1394  - 11.28 20.56 32.37 45.49 48.24 38.17 1:19.58 3:01.26  1393  - 11.29 20.57 32.39 45.51 48.27 38.19 1:19.62 3:01.36  1392 10.15  - 20.58 32.40 45.53 48.30 38.21 1:19.67 3:01.47  1391  - 11.30 20.59 32.42 45.56 48.33 38.24 1:19.72 3:01.58  1390 10.16 11.31 20.60 32.43 45.58 48.36 38.26 1:19.77 3:01.69  1389  -  -  - 32.45 45.60 48.39 38.28 1:19.81 3:01.79  1388 10.17 11.32 20.61 32.47 45.63 48.42 38.30 1:19.86 3:01.90  1387  - 11.33 20.62 32.48 45.65 48.45 38.32 1:19.91 3:02.01  1386  -

### Sprints

In [176]:
sprints = ['100m', '110mH', '200m', '300m', '400m', '400mH', '4x100m', '4x200m', '4x400m',]

In [192]:
def even_sprint_formatting(extracted_text, sprints):
    points = extracted_text.splitlines()[1].replace(' Points ',' Points  ').split(' Points  ')[1].replace(' - ','- ')
    points = re.split(' [0-9]{1,4} ',points)
    points_df = pd.DataFrame(columns = sprints, data=[['-'] + row.split() if len(row.split())==8 else row.split() for row in points[0:]]).replace('-',np.nan)
    return points_df

def odd_sprint_formatting(extracted_text,sprints):
    points = extracted_text.splitlines()[1].replace(' Points ',' Points  ').split(' Points  ')[1].replace(' - ','- ')
    points = re.split(' [0-9]{1,4} ',points)
    points_df = pd.DataFrame(columns = sprints, data=[['-'] + row.split() if len(row.split())==8 else row.split() for row in points[0:]]).replace('-',np.nan)
    return points_df


In [227]:
full_df = pd.DataFrame()

for i in range(189,217):  # M:(9,37) , W:(189,217)
    print(f'Formatting page {i}')
    page = reader.pages[i] 
    extracted_text = page.extract_text()
    if i%2 == 0: # even
        points_df = even_sprint_formatting(extracted_text,sprints)
    else:
        points_df = odd_sprint_formatting(extracted_text,sprints)

    full_df = pd.concat([full_df,points_df])


# Remove filler rows
full_df = full_df[full_df['100m']!='100m']
full_df = full_df[~full_df.isnull().all(1)]
full_df['Points'] = np.linspace(1400,1,1400).astype(int)
full_df

Formatting page 189
Formatting page 190
Formatting page 191
Formatting page 192
Formatting page 193
Formatting page 194
Formatting page 195
Formatting page 196
Formatting page 197
Formatting page 198
Formatting page 199
Formatting page 200
Formatting page 201
Formatting page 202
Formatting page 203
Formatting page 204
Formatting page 205
Formatting page 206
Formatting page 207
Formatting page 208
Formatting page 209
Formatting page 210
Formatting page 211
Formatting page 212
Formatting page 213
Formatting page 214
Formatting page 215
Formatting page 216


,100m,110mH,200m,300m,400m,400mH,4x100m,4x200m,4x400m,Points
1,10.12,11.24,20.51,32.27,45.35,48.07,38.04,1:19.29,3:00.61,1400
2,NaN,11.25,20.52,32.29,45.37,48.09,38.06,1:19.34,3:00.72,1399
3,NaN,NaN,NaN,32.31,45.40,48.12,38.08,1:19.39,3:00.83,1398
4,10.13,11.26,20.53,32.32,45.42,48.15,38.11,1:19.43,3:00.94,1397
5,NaN,11.27,20.54,32.34,45.44,48.18,38.13,1:19.48,3:01.04,1396
...,...,...,...,...,...,...,...,...,...,...
45,21.29,28.87,44.00,1:14.32,1:46.13,2:05.10,1:34.41,3:24.06,7:42.10,5
46,21.36,28.99,44.16,1:14.60,1:46.54,2:05.62,1:34.79,3:24.90,7:43.99,4
47,21.45,29.13,44.34,1:14.92,1:47.00,2:06.20,1:35.22,3:25.85,7:46.14,3
48,21.55,29.29,44.55,1:15.30,1:47.55,2:06.90,1:35.73,3:26.98,7:48.68,2


In [228]:
# full_df.to_csv('IAAF_W_Sprints.csv',index=False)
# df = pd.read_csv('IAAF_W_Sprints.csv')
# df

## Middle Distance

In [17]:
middle_distance = ['600m', '800m', '1000m', '1500m', 'Mile', '2000m',]

def middle_distance_formatting(extracted_text,middle_distance):    
    points = extracted_text.splitlines()[1].replace(' Points ',' Points  ').split(' Points  ')[1].replace(' - ','- ')
    points = re.split(' [0-9]{1,4} ',points)
    points_df = pd.DataFrame(columns = middle_distance, data=[['-'] + row.split() if len(row.split())==8 else row.split() for row in points[0:]]).replace('-',np.nan)
    return points_df
    

In [20]:
full_df = pd.DataFrame()

for i in range(219,247):  # M:(39,67) , W:(219,247) 
    print(f'Formatting page {i}')
    page = reader.pages[i] 
    extracted_text = page.extract_text()
    points_df = middle_distance_formatting(extracted_text, middle_distance)

    full_df = pd.concat([full_df,points_df])

# Remove filler rows
full_df = full_df[full_df['600m']!='600m']
full_df = full_df[~full_df.isnull().all(1)]
full_df['Points'] = np.linspace(1400,1,1400).astype(int)
full_df

Formatting page 219
Formatting page 220
Formatting page 221
Formatting page 222
Formatting page 223
Formatting page 224
Formatting page 225
Formatting page 226
Formatting page 227
Formatting page 228
Formatting page 229
Formatting page 230
Formatting page 231
Formatting page 232
Formatting page 233
Formatting page 234
Formatting page 235
Formatting page 236
Formatting page 237
Formatting page 238
Formatting page 239
Formatting page 240
Formatting page 241
Formatting page 242
Formatting page 243
Formatting page 244
Formatting page 245
Formatting page 246


,600m,800m,1000m,1500m,Mile,2000m,Points
1,1:15.62,1:47.35,2:18.56,3:36.77,3:53.34,4:55.11,1400
2,1:15.66,1:47.40,2:18.62,3:36.88,3:53.46,4:55.28,1399
3,1:15.70,1:47.45,2:18.69,3:37.00,3:53.58,4:55.44,1398
4,1:15.74,1:47.50,2:18.76,3:37.11,3:53.71,4:55.60,1397
5,1:15.78,1:47.55,2:18.83,3:37.23,3:53.83,4:55.76,1396
...,...,...,...,...,...,...,...
45,2:57.52,4:01.47,5:18.55,8:40.68,9:19.28,12:02.81,5
46,2:58.20,4:02.37,5:19.76,8:42.72,9:21.47,12:05.68,4
47,2:58.98,4:03.39,5:21.13,8:45.03,9:23.95,12:08.94,3
48,2:59.90,4:04.60,5:22.76,8:47.78,9:26.89,12:12.80,2


In [22]:
# full_df.to_csv('IAAF_F_Middle_Distance.csv',index=False)
# df = pd.read_csv('IAAF_F_Middle_Distance.csv')
# df

## Long Distance

In [30]:
long_distance = ['2000m SC', '3000m', '3000m SC', '2 Miles', '5000m', '10000m',]

def long_distance_formatting(extracted_text,long_distance):  
    extracted_text = extracted_text.replace('2000m SC','2000m_SC').replace('3000m SC', '3000m_SC').replace('2 Miles', '2_Miles') # Do this for all events - no gaps!  
    points = extracted_text.splitlines()[1].replace(' Points ',' Points  ').split(' Points  ')[1].replace(' - ','- ')
    points = re.split(' [0-9]{1,4} ',points)
    points_df = pd.DataFrame(columns = long_distance, data=[['-'] + row.split() if len(row.split())==8 else row.split() for row in points[0:]]).replace('-',np.nan)
    return points_df

In [36]:
full_df = pd.DataFrame()

for i in range(69,97):  # M:(69,97) , W:(249,277)
    print(f'Formatting page {i}')
    page = reader.pages[i] 
    extracted_text = page.extract_text()
    points_df = long_distance_formatting(extracted_text, long_distance)

    full_df = pd.concat([full_df,points_df])

# Remove filler rows
full_df = full_df[full_df['3000m']!='3000m']
full_df = full_df[~full_df.isnull().all(1)]
full_df['Points'] = np.linspace(1400,1,1400).astype(int)
full_df

Formatting page 69
Formatting page 70
Formatting page 71
Formatting page 72
Formatting page 73
Formatting page 74
Formatting page 75
Formatting page 76
Formatting page 77
Formatting page 78
Formatting page 79
Formatting page 80
Formatting page 81
Formatting page 82
Formatting page 83
Formatting page 84
Formatting page 85
Formatting page 86
Formatting page 87
Formatting page 88
Formatting page 89
Formatting page 90
Formatting page 91
Formatting page 92
Formatting page 93
Formatting page 94
Formatting page 95
Formatting page 96


,2000m SC,3000m,3000m SC,2 Miles,5000m,10000m,Points
1,4:50.06,7:05.53,7:30.46,7:38.54,12:10.09,25:15.44,1400
2,4:50.19,7:05.68,7:30.66,7:38.70,12:10.35,25:16.03,1399
3,4:50.32,7:05.83,7:30.86,7:38.86,12:10.60,25:16.61,1398
4,4:50.46,7:05.98,7:31.07,7:39.02,12:10.85,25:17.20,1397
5,4:50.59,7:06.13,7:31.27,7:39.17,12:11.11,25:17.78,1396
...,...,...,...,...,...,...,...
45,10:37.89,13:35.23,16:25.96,14:38.13,23:17.57,50:52.31,5
46,10:40.22,13:37.84,16:29.55,14:40.94,23:22.05,51:02.62,4
47,10:42.87,13:40.81,16:33.63,14:44.14,23:27.13,51:14.33,3
48,10:46.01,13:44.33,16:38.47,14:47.93,23:33.16,51:28.21,2


In [53]:
# full_df.to_csv('IAAF_W_Long_Distance.csv',index=False)
# df = pd.read_csv('IAAF_W_Long_Distance.csv')
# df

## Road Running

In [45]:
road_running = ['10 km','15 km','10 Miles','20 km', 'HM','25 km','30 km','Marathon','100 km',]

def road_running_formatting(extracted_text,road_running):    
    extracted_text = extracted_text.replace('0 km','0_km').replace('5 km','5_km').replace('10 Miles','10_Miles')
    points = extracted_text.splitlines()[1].replace(' Points ',' Points  ').split(' Points  ')[1].replace(' - ','- ')
    points = re.split(' [0-9]{1,4} ',points)
    points_df = pd.DataFrame(columns = road_running, data=[['-'] + row.split() if len(row.split())==8 else row.split() for row in points[0:]]).replace('-',np.nan)
    return points_df

In [50]:
full_df = pd.DataFrame()

for i in range(279,307):  # M:(99,127) , W:(279,307)
    print(f'Formatting page {i}')
    page = reader.pages[i] 
    extracted_text = page.extract_text()
    points_df = road_running_formatting(extracted_text, road_running)

    full_df = pd.concat([full_df,points_df])

# Remove filler rows
full_df = full_df[full_df['10 km']!='10_km']
full_df = full_df[~full_df.isnull().all(1)]
full_df['Points'] = np.linspace(1400,1,1400).astype(int)
full_df

Formatting page 279
Formatting page 280
Formatting page 281
Formatting page 282
Formatting page 283
Formatting page 284
Formatting page 285
Formatting page 286
Formatting page 287
Formatting page 288
Formatting page 289
Formatting page 290
Formatting page 291
Formatting page 292
Formatting page 293
Formatting page 294
Formatting page 295
Formatting page 296
Formatting page 297
Formatting page 298
Formatting page 299
Formatting page 300
Formatting page 301
Formatting page 302
Formatting page 303
Formatting page 304
Formatting page 305
Formatting page 306


,10 km,15 km,10 Miles,20 km,HM,25 km,30 km,Marathon,100 km,Points
1,27:45,42:11,45:23,56:51,1:00:02,1:11:47,1:26:54,2:04:20,5:52:57,1400
2,27:46,42:13,45:25,56:53,1:00:04,1:11:50,1:26:58,2:04:26,5:53:11,1399
3,27:47,42:14,45:27,56:55,1:00:06,1:11:53,1:27:01,2:04:31,5:53:25,1398
4,27:48,42:16,45:29,56:57,1:00:09,1:11:56,1:27:05,2:04:37,5:53:40,1397
5,27:49,42:17,45:30,56:59,1:00:11,1:11:59,1:27:08,2:04:42,5:53:54,1396
...,...,...,...,...,...,...,...,...,...,...
45,1:12:10,1:50:43,1:59:16,2:30:01,2:38:43,3:14:35,4:02:10,6:04:43,16:20:08,5
46,1:12:28,1:51:11,1:59:46,2:30:39,2:39:23,3:15:25,4:03:13,6:06:20,16:24:20,4
47,1:12:48,1:51:42,2:00:19,2:31:21,2:40:08,3:16:21,4:04:24,6:08:09,16:29:07,3
48,1:13:12,1:52:19,2:00:59,2:32:12,2:41:01,3:17:27,4:05:48,6:10:20,16:34:47,2


In [52]:
# full_df.to_csv('IAAF_W_Road_Running.csv',index=False)
# df = pd.read_csv('IAAF_W_Road_Running.csv')
# df

## Concatenate the tables for easier comparisons

Just want a wide table for each of Male and Female

In [2]:
import pandas as pd

In [32]:
# s = pd.read_csv('IAAF_W_Sprints.csv')
# m = pd.read_csv('IAAF_F_Middle_Distance.csv')
# l = pd.read_csv('IAAF_W_Long_Distance.csv')
# r = pd.read_csv('IAAF_W_Road_Running.csv')

# pd.read_csv('IAAF_F.csv')

In [33]:
# iaaf = pd.concat([s,m,l,r],axis=1).T.drop_duplicates().T
# new_col_order = ['Points'] + [col for col in iaaf.columns if col != 'Points']

# iaaf = iaaf.reindex(columns=new_col_order)
# iaaf
# iaaf.to_csv('IAAF_F.csv',index=False)

## Convert to Seconds

In [11]:
import pandas as pd
from datetime import datetime

In [12]:
## Load in the IAAF points tables
iaaf_m = pd.read_csv('IAAF_M.csv').ffill()
iaaf_f = pd.read_csv('IAAF_F.csv').ffill()

iaaf_f

,Points,100m,110mH,200m,300m,400m,400mH,4x100m,4x200m,4x400m,...,10000m,10 km,15 km,10 Miles,20 km,HM,25 km,30 km,Marathon,100 km
0,1400,10.12,11.24,20.51,32.27,45.35,48.07,38.04,1:19.29,3:00.61,...,27:20.35,27:45,42:11,45:23,56:51,1:00:02,1:11:47,1:26:54,2:04:20,5:52:57
1,1399,10.12,11.25,20.52,32.29,45.37,48.09,38.06,1:19.34,3:00.72,...,27:21.37,27:46,42:13,45:25,56:53,1:00:04,1:11:50,1:26:58,2:04:26,5:53:11
2,1398,10.12,11.25,20.52,32.31,45.40,48.12,38.08,1:19.39,3:00.83,...,27:22.39,27:47,42:14,45:27,56:55,1:00:06,1:11:53,1:27:01,2:04:31,5:53:25
3,1397,10.13,11.26,20.53,32.32,45.42,48.15,38.11,1:19.43,3:00.94,...,27:23.42,27:48,42:16,45:29,56:57,1:00:09,1:11:56,1:27:05,2:04:37,5:53:40
4,1396,10.13,11.27,20.54,32.34,45.44,48.18,38.13,1:19.48,3:01.04,...,27:24.44,27:49,42:17,45:30,56:59,1:00:11,1:11:59,1:27:08,2:04:42,5:53:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,5,21.29,28.87,44.00,1:14.32,1:46.13,2:05.10,1:34.41,3:24.06,7:42.10,...,1:12:09.10,1:12:10,1:50:43,1:59:16,2:30:01,2:38:43,3:14:35,4:02:10,6:04:43,16:20:08
1396,4,21.36,28.99,44.16,1:14.60,1:46.54,2:05.62,1:34.79,3:24.90,7:43.99,...,1:12:27.14,1:12:28,1:51:11,1:59:46,2:30:39,2:39:23,3:15:25,4:03:13,6:06:20,16:24:20
1397,3,21.45,29.13,44.34,1:14.92,1:47.00,2:06.20,1:35.22,3:25.85,7:46.14,...,1:12:47.62,1:12:48,1:51:42,2:00:19,2:31:21,2:40:08,3:16:21,4:04:24,6:08:09,16:29:07
1398,2,21.55,29.29,44.55,1:15.30,1:47.55,2:06.90,1:35.73,3:26.98,7:48.68,...,1:13:11.91,1:13:12,1:52:19,2:00:59,2:32:12,2:41:01,3:17:27,4:05:48,6:10:20,16:34:47


In [13]:
def format_iaaf_tables(iaaf):
    """
    Function to convert all iaaf times into total seconds. 
    E.g 
    1:18.43 -> 78.43 (seconds)
    31:12.3 -> 1872.3 (seconds)

    If else conditions to manage multiple datetime formats in the iaaf tables
    """
    str_cols = iaaf.loc[:,iaaf.dtypes == object].columns

    for col in str_cols:
        if col in['300m', '400m', '400mH', '4x100m', '4x200m', '4x400m', '600m', '800m', '1000m',
        '1500m', 'Mile', '2000m', '2000m SC', '3000m', '3000m SC', '2 Miles',
        '5000m','10000m']:
            # These columns have format S.ff or M:SS.ff
            iaaf[col] = iaaf[col].apply(lambda x: datetime.strptime(x,"%S.%f") if len(x) == 5 else (datetime.strptime(x,"%H:%M:%S.%f") if len(x) > 9 else datetime.strptime(x,"%M:%S.%f")))
        else:
            # But the other columns have format M:S or HH:MM:S
            iaaf[col] = iaaf[col].apply(lambda x: datetime.strptime(x,"%M:%S") if len(x) == 5 else datetime.strptime(x,"%H:%M:%S"))
        # Total second calculation can be applied now formats are aligned
        iaaf[col] = iaaf[col].apply(lambda x: (x - datetime(1900, 1, 1)).total_seconds())

    return iaaf

print('m')
iaaf_m = format_iaaf_tables(iaaf_m)
print('f')
iaaf_f = format_iaaf_tables(iaaf_f)

# Use this code to look for incorrect data types
# df = iaaf['3000m'].map(type)#  == "str"
# for i in range(len(df)):
#             if df[i] ==str:
#                     pass
#             else:
#                     print(i)

m
f


In [15]:
iaaf_f # all times now in seconds

,Points,100m,110mH,200m,300m,400m,400mH,4x100m,4x200m,4x400m,...,10000m,10 km,15 km,10 Miles,20 km,HM,25 km,30 km,Marathon,100 km
0,1400,10.12,11.24,20.51,32.27,45.35,48.07,38.04,79.29,180.61,...,1640.35,1665.0,2531.0,2723.0,3411.0,3602.0,4307.0,5214.0,7460.0,21177.0
1,1399,10.12,11.25,20.52,32.29,45.37,48.09,38.06,79.34,180.72,...,1641.37,1666.0,2533.0,2725.0,3413.0,3604.0,4310.0,5218.0,7466.0,21191.0
2,1398,10.12,11.25,20.52,32.31,45.40,48.12,38.08,79.39,180.83,...,1642.39,1667.0,2534.0,2727.0,3415.0,3606.0,4313.0,5221.0,7471.0,21205.0
3,1397,10.13,11.26,20.53,32.32,45.42,48.15,38.11,79.43,180.94,...,1643.42,1668.0,2536.0,2729.0,3417.0,3609.0,4316.0,5225.0,7477.0,21220.0
4,1396,10.13,11.27,20.54,32.34,45.44,48.18,38.13,79.48,181.04,...,1644.44,1669.0,2537.0,2730.0,3419.0,3611.0,4319.0,5228.0,7482.0,21234.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,5,21.29,28.87,44.00,74.32,106.13,125.10,94.41,204.06,462.10,...,4329.10,4330.0,6643.0,7156.0,9001.0,9523.0,11675.0,14530.0,21883.0,58808.0
1396,4,21.36,28.99,44.16,74.60,106.54,125.62,94.79,204.90,463.99,...,4347.14,4348.0,6671.0,7186.0,9039.0,9563.0,11725.0,14593.0,21980.0,59060.0
1397,3,21.45,29.13,44.34,74.92,107.00,126.20,95.22,205.85,466.14,...,4367.62,4368.0,6702.0,7219.0,9081.0,9608.0,11781.0,14664.0,22089.0,59347.0
1398,2,21.55,29.29,44.55,75.30,107.55,126.90,95.73,206.98,468.68,...,4391.91,4392.0,6739.0,7259.0,9132.0,9661.0,11847.0,14748.0,22220.0,59687.0


In [19]:
# iaaf_m.to_csv('IAAF_M_seconds.csv',index=False)
# iaaf_f.to_csv('IAAF_F_seconds.csv',index=False)